# Grover Search algorithm (Q# code)

This notebook implements quantum Grover's Search algorithm in Q#.

See [here](https://tsmatz.wordpress.com/2019/03/12/quantum-computing-grover-algorithm-programming/) for mathematical background.

*back to [index](https://github.com/tsmatz/quantum-algorithms-qsharp)*

In [1]:
import qsharp

Preparing Q# environment...
.

In [2]:
# declare function
ApplyQuantumSearch: any = None

In [3]:
%%qsharp
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

operation ApplyQuantumSearch (nIterations : Int, nDatabaseQubits : Int) : (Result, Result[]) {
    use (markedQubit, databaseRegister) = (Qubit(), Qubit[nDatabaseQubits]) {
        QuantumSearch(nIterations, markedQubit, databaseRegister);

        // Measure the marked qubit.
        // On success, this should be |1〉.
        let resultSuccess = M(markedQubit);

        // Measure the state of the database register.
        // On success, this should be |1〉 |1〉 ... |1〉.
        let resultElement = MultiM(databaseRegister);

        // Reset all qubits to the |0〉 state.
        Reset(markedQubit);
        for idxResult in 0 .. nDatabaseQubits - 1 {
            Reset(databaseRegister[idxResult]);
        }

        return (resultSuccess, resultElement);
    }
}

operation QuantumSearch (nIterations : Int, markedQubit : Qubit, databaseRegister : Qubit[]) : Unit {
    StatePreparationOracle(markedQubit, databaseRegister);
    for idx in 0 .. nIterations - 1 {
        ReflectMarked(markedQubit);
        ReflectStart(markedQubit, databaseRegister);
    }
}

operation StatePreparationOracle (markedQubit : Qubit, databaseRegister : Qubit[]) : Unit is Adj + Ctl{
    UniformSuperpositionOracle(databaseRegister);
    DatabaseOracle(markedQubit, databaseRegister);
}

operation UniformSuperpositionOracle (databaseRegister : Qubit[]) : Unit is Adj + Ctl {
    body (...) {
        let nQubits = Length(databaseRegister);
        for idxQubit in 0 .. nQubits - 1 {
            H(databaseRegister[idxQubit]);
        }
    }
}

operation DatabaseOracle (markedQubit : Qubit, databaseRegister : Qubit[]) : Unit is Adj + Ctl {
    Controlled X(databaseRegister, markedQubit);
}

operation ReflectMarked (markedQubit : Qubit) : Unit {
    R1(PI(), markedQubit);
}

operation ReflectStart (markedQubit : Qubit, databaseRegister : Qubit[]) : Unit {
    Adjoint StatePreparationOracle(markedQubit, databaseRegister);
    ReflectZero([markedQubit] + databaseRegister);
    StatePreparationOracle(markedQubit, databaseRegister);
}

operation ReflectZero (allQubits : Qubit[]) : Unit {
    let nQubits = Length(allQubits);
    for idxQubit in 0 .. nQubits - 1 {
        X(allQubits[idxQubit]);
    }
    Controlled Z(allQubits[1 .. nQubits - 1], allQubits[0]);
    for idxQubit in 0 .. nQubits - 1 {
        X(allQubits[idxQubit]);
    }
}

In [4]:
db_qbit_len = 6;
iterations = 3;
results = ApplyQuantumSearch.simulate(
    nIterations=iterations,
    nDatabaseQubits=db_qbit_len)
markedQubit = results[0]
databaseRegister = results[1]
if markedQubit == 1:
    print("Succeed !")
    print("Found database index : {}".format(databaseRegister))
else:
    print("Failed !")

Succeed !
Found database index : [1, 1, 1, 1, 1, 1]
